# JOINS 

This jupyter is to join the datasets with their 
one to many notes files.

In [1]:
import string
import bz2
from os import listdir
from os.path import isfile, join
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import nltk.corpus  
import time
import math
import re
import spacy
from functools import reduce
from nltk.text import Text
from nltk.corpus import stopwords
from itertools import combinations
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import TextBlob
from nltk.stem.wordnet import WordNetLemmatizer
# from keras.preprocessing import text, sequence
# from keras.utils import to_categorical
# from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPool2D, GlobalMaxPool3D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation
# from keras.models import Model, Sequential
# from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
plt.style.use('fivethirtyeight')


In [2]:
os.getcwd()

'C:\\Users\\t.cri.kcheung\\capstone_nlp\\src\\joins'

In [3]:
# get all files in subfolder
mypath = "Z:/FINAL OUTPUT v2/"
onlyfiles = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
onlyfiles

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_movedfromcohort1.csv']

In [5]:
onlyfiles

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

# Modular

In [5]:
onlyfiles_sorted = sorted(onlyfiles, key=lambda x:(x!="PATIENT_COHORT", x))

In [6]:
onlyfiles_sorted_edit = list( onlyfiles_sorted[i] for i in [0,4,6,8,10,12] )
onlyfiles_sorted_edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv']

### Prior to loading in lets check for j84 in part 2

In [7]:
onlyfiles_sorted[11]

'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv'

In [8]:
cohort2_icd_check = pd.read_csv(onlyfiles_sorted[11],sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# get list of columns with dx
dx_cols = [col for col in cohort2_icd_check.columns if 'dx_' in col]
dx_cols[0]

'dx_1'

In [10]:
# all icd codes list
icd_codes_list = ['495','4951','4952','4953','4954','4955','4956','4957','4958','4959','496','515','517','5171','5172','5173','5178','516','5160','5161','5162','5163','51630','51631','51632','51633','51634','51635','51636','51637','5164','5165','5166','51661','51662','51663','51664','51669','5168','5169','J670','J671','J672','J673','J674','J675','J676','J677','J678','J679','J6710','J8410','J8489','J17','M3481','J99','J8401','J8402','J8403','J84111','J84112','J84113','J84114','J84115','J84116','J84117','J8481','J8482','J8483','J84841','J84842','J84843','J84844','J84845','J84848','J8409','J849']
pattern = '|'.join(icd_codes_list)


In [11]:
# extract all
icd_subset = []

for col in dx_cols:
#     print(col)
    tmpdf = cohort2_icd_check[cohort2_icd_check[col].str.contains(pattern,na=False)]
    icd_subset.append(tmpdf)
# print(icd_subset)
    

In [12]:
icd_subset_df = icd_subset[0]

In [13]:
len(np.unique(icd_subset_df.PATIENT_ID))

2243

# Now i have all the patients with the relevant icd codes. Around 2k unique of them.

We need to extract all of the relevant records into their own df. Let's tweak the merging into just those of which that are in the lsit of patients.

In [16]:
onlyfiles_sorted

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv']

In [26]:
cohort2move = pd.read_csv(onlyfiles_sorted[3],sep="|")
cohort2move.head()

PATIENT_ID  AGE_AT_LAST_ENCOUNTER gender                    race  \
0          49                   79.0      M                   White   
1          99                   86.0      F                   White   
2         104                   84.0      M                   White   
3         114                   66.0      F                   White   
4         118                   92.0      F  Black/African-American   

                ETHNICITY DATE_OF_DEATH_D  
0  Not Hispanic or Latino             NaN  
1  Not Hispanic or Latino             NaN  
2  Not Hispanic or Latino             NaN  
3  Not Hispanic or Latino             NaN  
4  Not Hispanic or Latino             NaN

In [18]:
cohort2movedf = cohort2move[cohort2move.PATIENT_ID.isin(icd_subset_df.PATIENT_ID)]

In [19]:
cohort2movedf.to_csv(join(mypath,"01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv"),sep="|",index=False)

### Prior to loading in the second cohort let's randomly sample in 10k rows


In [32]:
cohort2move.head()

NameError: name 'cohort2move' is not defined

In [28]:
cohort2removeddf = cohort2move[~cohort2move.PATIENT_ID.isin(icd_subset_df.PATIENT_ID)]

In [29]:
cohort2sampdf = cohort2removeddf.sample(n=30000,random_state=10,replace=False)
cohort2sampdf.shape

(30000, 6)

In [271]:
cohort2sampdf.to_csv(join(mypath,"01_PATIENT_COHORT_2_NEW.csv"),sep="|",index=False)

In [31]:
del cohort2move, cohort2_icd_check,cohort2move

NameError: name 'cohort2_icd_check' is not defined

# Next step we need to create a join list with just the ids in moved from cohort2 set

Then we can merge them into a consolidated cohort1

In [34]:
onlyfiles2 = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [35]:
onlyfiles2

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

In [36]:
onlyfiles2edit = list( onlyfiles2[i] for i in [0, 6, 7, 13,14,15] )
# onlyfiles2edit = list( onlyfiles2[i] for i in [0,15] )

In [37]:
onlyfiles2edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

In [38]:
# test
def test(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientids)]
            return(df)

In [39]:
patientlist = [114]

In [40]:
test(onlyfiles2edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv


patient_id  age_at_last_encounter gender   race               ethnicity  \
0         114                   66.0      F  White  Not Hispanic or Latino   

  date_of_death_d  
0             NaN

### Edited fx

For mem purposes let's amend each read in to be just where it matches the patient id string

In [41]:
patientlist = np.unique(icd_subset_df['PATIENT_ID'])
patientlist[0:5]

array([ 114,  431,  672, 1408, 1549], dtype=int64)

In [44]:
frame = pd.DataFrame()
list_ = []

def readin_select(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)]
            list_.append(df)

In [45]:
readin_select(onlyfiles2edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv
added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv


In [46]:
list_[0].iloc[1:5,:]

patient_id  age_at_last_encounter gender                    race  \
1         431                   88.0      F  Black/African-American   
2         672                   80.0      M    Asian/Mideast Indian   
3        1408                   84.0      F  Black/African-American   
4        1549                   80.0      F  Black/African-American   

                ethnicity date_of_death_d  
1  Not Hispanic or Latino             NaN  
2  Not Hispanic or Latino             NaN  
3  Not Hispanic or Latino             NaN  
4  Not Hispanic or Latino             NaN

In [56]:
list_[5].iloc[1:5,:]

patient_id  encounter_id  order_proc_id_deid        ordering_date_off  \
26     1766157       3759747             1002716  2012-11-28 00:00:00.000   
53     1380083       3938864              800327  2012-10-24 00:00:00.000   
62     1366083       3939557              737500  2013-06-21 00:00:00.000   
63     1366083       3939557             1260359  2013-06-21 00:00:00.000   

                      exam_type  
26   CT Chest & Upper Abdomen w  
53  CT Upper Abdomen & Pelvis w  
62   CT Chest & Upper Abdomen w  
63   CT Chest & Upper Abdomen w

In [47]:
len(list_[5])

59663

In [58]:
# spot check on patient ids 
list_[5][list_[5].patient_id==1408]
list_[0][list_[0].patient_id==1380083]


patient_id  age_at_last_encounter gender                    race  \
879     1380083                   65.0      F  Black/African-American   

                  ethnicity          date_of_death_d  
879  Not Hispanic or Latino  2014-10-13 00:00:00.000

# Correctly appending in function

Let's now see if we can use dask to improve the join functionality of the other datasets given memory issues.

In [59]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

df1 = list_[0]
df2 = list_[5]

In [60]:
print(df1.head())
print(len(df1))

   patient_id  age_at_last_encounter gender                    race  \
0         114                   66.0      F                   White   
1         431                   88.0      F  Black/African-American   
2         672                   80.0      M    Asian/Mideast Indian   
3        1408                   84.0      F  Black/African-American   
4        1549                   80.0      F  Black/African-American   

                ethnicity date_of_death_d  
0  Not Hispanic or Latino             NaN  
1  Not Hispanic or Latino             NaN  
2  Not Hispanic or Latino             NaN  
3  Not Hispanic or Latino             NaN  
4  Not Hispanic or Latino             NaN  
2243


In [61]:
print(df2.head())
print(len(df2))

    patient_id  encounter_id  order_proc_id_deid        ordering_date_off  \
25     1766157       3759747              706804  2012-11-28 00:00:00.000   
26     1766157       3759747             1002716  2012-11-28 00:00:00.000   
53     1380083       3938864              800327  2012-10-24 00:00:00.000   
62     1366083       3939557              737500  2013-06-21 00:00:00.000   
63     1366083       3939557             1260359  2013-06-21 00:00:00.000   

                      exam_type  
25   CT Chest & Upper Abdomen w  
26   CT Chest & Upper Abdomen w  
53  CT Upper Abdomen & Pelvis w  
62   CT Chest & Upper Abdomen w  
63   CT Chest & Upper Abdomen w  
59663


In [62]:
mergetest = dd.merge(df1, df2, how='outer', on=['patient_id'])

In [63]:
mergetest

patient_id  age_at_last_encounter gender                    race  \
0             114                   66.0      F                   White   
1             114                   66.0      F                   White   
2             114                   66.0      F                   White   
3             114                   66.0      F                   White   
4             114                   66.0      F                   White   
5             114                   66.0      F                   White   
6             114                   66.0      F                   White   
7             114                   66.0      F                   White   
8             114                   66.0      F                   White   
9             114                   66.0      F                   White   
10            114                   66.0      F                   White   
11            114                   66.0      F                   White   
12            114                   66.0      F                   White   
13            114                   66.0      F                   White   
14            114                   66.0      F                   White   
15            114                   66.0      F                   White   
16            114                   66.0      F                   White   
17            114                   66.0      F                   White   
18            114                   66.0      F                   White   
19            114                   66.0      F                   White   
20            114                   66.0      F                   White   
21            114                   66.0      F                   White   
22            114                   66.0      F                   White   
23            114                   66.0      F                   White   
24            114                   66.0      F                   White   
25            114                   66.0      F                   White   
26            114                   66.0      F                   White   
27            114                   66.0      F                   White   
28            114                   66.0      F                   White   
29            114                   66.0      F                   White   
...           ...                    ...    ...                     ...   
59633     2740460                   40.0      M                   White   
59634     2740460                   40.0      M                   White   
59635     2740460                   40.0      M                   White   
59636     2740460                   40.0      M                   White   
59637     2740460                   40.0      M                   White   
59638     2740460                   40.0      M                   White   
59639     2740460                   40.0      M                   White   
59640     2740460                   40.0      M                   White   
59641     2740460                   40.0      M                   White   
59642     2740460                   40.0      M                   White   
59643     2779581                   74.0      M                   White   
59644     2779581                   74.0      M                   White   
59645     2779581                   74.0      M                   White   
59646     2849686                   56.0      M  Black/African-American   
59647     2849686                   56.0      M  Black/African-American   
59648     2849686                   56.0      M  Black/African-American   
59649     2959602                   64.0      M  Black/African-American   
59650     2959602                   64.0      M  Black/African-American   
59651     2959602                   64.0      M  Black/African-American   
59652     2959602                   64.0      M  Black/African-American   
59653     2959602                   64.0      M  Black/African-American   
59654     2959602               

Let's redo the function now with the dd read in and merge.

In [89]:
frame = pd.DataFrame()
list_ = []

def readin_select_full(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)].dropna(axis='columns',how='all')
            list_.append(df)
#     suffix = ('_patient', '_encounter', '_encounter_provider', '_encounter_social','_dx_info','_radiology')
    suffix = ('_patient','_radiology')
    for i in range(0,len(list_)):
        print("iteration number",i)
        tmp1 = pd.DataFrame(list_[i].iloc[:,0])
        print("suffix : ",suffix[i])
        list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
        list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
#     frame = reduce(lambda x, y: pd.merge(x, y, on = 'patient_id',how="left"), list_)
    frame = reduce(lambda x, y: dd.merge(x, y, on = ['patient_id'],how="left"), list_)
    print(len(frame))
    return(frame)

In [69]:
onlyfiles3edit = list( onlyfiles2[i] for i in [0,15] )
onlyfiles3edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

In [74]:
readin_select_full(onlyfiles3edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv
iteration number 0
suffix :  _patient
iteration number 1
suffix :  _radiology
59663


patient_id  age_at_last_encounter_patient gender_patient  \
0             114                           66.0              F   
1             114                           66.0              F   
2             114                           66.0              F   
3             114                           66.0              F   
4             114                           66.0              F   
5             114                           66.0              F   
6             114                           66.0              F   
7             114                           66.0              F   
8             114                           66.0              F   
9             114                           66.0              F   
10            114                           66.0              F   
11            114                           66.0              F   
12            114                           66.0              F   
13            114                           66.0              F   
14            114                           66.0              F   
15            114                           66.0              F   
16            114                           66.0              F   
17            114                           66.0              F   
18            114                           66.0              F   
19            114                           66.0              F   
20            114                           66.0              F   
21            114                           66.0              F   
22            114                           66.0              F   
23            114                           66.0              F   
24            114                           66.0              F   
25            114                           66.0              F   
26            114                           66.0              F   
27            114                           66.0              F   
28            114                           66.0              F   
29            114                           66.0              F   
...           ...                            ...            ...   
59633     2740460                           40.0              M   
59634     2740460                           40.0              M   
59635     2740460                           40.0              M   
59636     2740460                           40.0              M   
59637     2740460                           40.0              M   
59638     2740460                           40.0              M   
59639     2740460                           40.0              M   
59640     2740460                           40.0              M   
59641     2740460                           40.0              M   
59642     2740460                           40.0              M   
59643     2779581                           74.0              M   
59644     2779581                           74.0              M   
59645     2779581                           74.0              M   
59646     2849686                           56.0              M   
59647     2849686                           56.0              M   
59648     2849686                           56.0              M   
59649     2959602                           64.0              M   
59650     2959602                           64.0              M   
59651     2959602                           64.0              M   
59652     2959602                           64.0              M   
59653     2959602                           64.0              M   
59654     2959602                           64.0              M   
59655     2959602                           64.0              M   
59656     2959602                           64.0              M   
59657     2959602                           64.0              M   
59658     2959602                           64.0              M   
59659     2959602                           64.0              M   
59660     2959602                           64.0      

Looks good now. Full run. Comment out the suffix.

In [82]:
list_[4].iloc[0:5,:]

patient_id  encounter_id_dx_info dx_1_dx_info  \
3          114               9494644         J849   
15         114               6747726       786.50   
21         114               5066736        786.2   
26         672               9011341        R0602   
34         114               9495002         J849   

                              dx_desc_1_dx_info  dx1_rank_dx_info  \
3   INTERSTITIAL PULMONARY DISEASE, UNSPECIFIED               1.0   
15                       UNSPECIFIED CHEST PAIN               1.0   
21                                        COUGH               1.0   
26                          SHORTNESS OF BREATH               1.0   
34  INTERSTITIAL PULMONARY DISEASE, UNSPECIFIED               1.0   

   dx_2_dx_info                                 dx_desc_2_dx_info  \
3          J849       INTERSTITIAL PULMONARY DISEASE, UNSPECIFIED   
15       793.19  OTHER NONSPECIFIC ABNORMAL FINDING OF LUNG FIELD   
21        305.1                 NONDEPENDENT TOBACCO USE DISORDER   
26         R079                           CHEST PAIN, UNSPECIFIED   
34        J9601            ACUTE RESPIRATORY FAILURE WITH HYPOXIA   

    dx2_rank_dx_info dx_3_dx_info                          dx_desc_3_dx_info  \
3                2.0          NaN                                        NaN   
15               2.0          NaN                                        NaN   
21               2.0          NaN                                        NaN   
26               2.0         J449  CHRONIC OBSTRUCTIVE PULMONARY DISEASE USP   
34               2.0         J189            PNEUMONIA, UNSPECIFIED ORGANISM   

          ...          dx83_rank_dx_info dx_84_dx_info dx_desc_84_dx_info  \
3         ...                        NaN           NaN                NaN   
15        ...                        NaN           NaN                NaN   
21        ...                        NaN           NaN                NaN   
26        ...                        NaN           NaN                NaN   
34        ...                        NaN           NaN                NaN   

    dx84_rank_dx_info dx_85_dx_info dx_desc_85_dx_info  dx85_rank_dx_info  \
3                 NaN           NaN                NaN                NaN   
15                NaN           NaN                NaN                NaN   
21                NaN           NaN                NaN                NaN   
26                NaN           NaN                NaN                NaN   
34                NaN           NaN                NaN                NaN   

   dx_86_dx_info dx_desc_86_dx_info  dx86_rank_dx_info  
3            NaN                NaN                NaN  
15           NaN                NaN                NaN  
21           NaN                NaN                NaN  
26           NaN                NaN                NaN  
34           NaN                NaN                NaN  

[5 rows x 260 columns]

# Merge first grouping

In [282]:
onlyfiles1 = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [283]:
onlyfiles1

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_movedfromcohort1.csv']

In [284]:
# ORDER NOW MATTERS
onlyfiles1 = list( onlyfiles1[i] for i in [2,11,3,12] )
onlyfiles1

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv']

In [286]:
patientlist = np.unique(pd.read_csv(onlyfiles1[0],sep="|").PATIENT_ID)
patientlist[0:5]

array([ 32124,  74912,  81545,  93110, 118647], dtype=int64)

# Function def

Function to populate a list of dfs.

In [287]:
def get_list(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)]
            list_.append(df)
    suffix = ('_patient', '_encounter','_dx_info','radiology')
    for i in range(0,len(list_)):
        print("iteration number",i)
        print("suffix : ",suffix[i])
        if i == 0:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0])
            list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)
        else:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0:2])
            list_[i] = list_[i].iloc[:,2:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)

In [265]:
def merge_lists(data):
    for i in range(0,len(list_)):
        print(i)
        if i == 1:
            print("merging first list slice: ", i) 
            print(data[i].iloc[0:1,:])
            df1 = data[0]
            df2 = data[i]
            holdingdf = pd.merge(df1, df2, how='left', on=['patient_id'])
        if i > 1:
            print("merging next list slice: ", i)
            print(data[i].iloc[0:1,:])
            df1 = holdingdf
            df2 = data[i]
            holdingdf = pd.merge(df1, df2, how='left', on=['patient_id','encounter_id'])
    return(holdingdf)

# Now run list and merge functions on pop one

In [288]:
list_ = []
get_list(onlyfiles1,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv
iteration number 0
suffix :  _patient
dimensions :  (126, 6)
iteration number 1
suffix :  _encounter
dimensions :  (250, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (250, 41)
iteration number 3
suffix :  radiology
dimensions :  (133, 5)


In [289]:
cohort1dta = merge_lists(list_)

0
1
merging first list slice:  1
   patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
0       32124       5039107                 O                Completed   

  visit_status_2_encounter visit_status_3_encounter  visit_status_4_encounter  \
0                      NaN                      NaN                       NaN   

      adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
0  2012-08-01 00:00:00.000  2012-08-01 00:00:00.000  2012-08-01 00:00:00.000   

      end_date_d_encounter admit_source_1_encounter admit_source_2_encounter  \
0  2012-08-01 00:00:00.000                      NaN                      NaN   

   admit_source_3_encounter disch_disp_1_encounter  disch_disp_2_encounter  \
0                       NaN                    NaN                     NaN   

   disch_disp_3_encounter  zip_d_encounter enc_fin_class_encounter  
0                     NaN           100058                Medicare  
2
merging next list slice:  2
   patie

In [290]:
cohort1dta.head()

patient_id  age_at_last_encounter_patient gender_patient  \
0       32124                           90.0              F   
1       74912                           63.0              F   
2       81545                           65.0              M   
3       93110                           47.0              F   
4      118647                           52.0              F   

             race_patient       ethnicity_patient  date_of_death_d_patient  \
0                   White  Not Hispanic or Latino                      NaN   
1  Black/African-American  Not Hispanic or Latino                      NaN   
2                   White  Not Hispanic or Latino  2015-06-21 00:00:00.000   
3  Black/African-American  Not Hispanic or Latino  2013-04-07 00:00:00.000   
4  Black/African-American  Not Hispanic or Latino                      NaN   

   encounter_id enc_eio_encounter visit_status_1_encounter  \
0       5039107                 O                Completed   
1       5237464                 O                Completed   
2       4429153                 O                Completed   
3       5226111                 O                Completed   
4       6862790                 O                Completed   

  visit_status_2_encounter           ...            dx11_rank_dx_info  \
0                      NaN           ...                          NaN   
1                      NaN           ...                          NaN   
2                 Canceled           ...                          NaN   
3                 Canceled           ...                          NaN   
4                      NaN           ...                          NaN   

   dx_12_dx_info dx_desc_12_dx_info dx12_rank_dx_info dx_13_dx_info  \
0            NaN                NaN               NaN           NaN   
1            NaN                NaN               NaN           NaN   
2            NaN                NaN               NaN           NaN   
3            NaN                NaN               NaN           NaN   
4            NaN                NaN               NaN           NaN   

  dx_desc_13_dx_info dx13_rank_dx_info order_proc_id_deidradiology  \
0                NaN               NaN                         NaN   
1                NaN               NaN                         NaN   
2                NaN               NaN                         NaN   
3                NaN               NaN                         NaN   
4                NaN               NaN                    178548.0   

   ordering_date_offradiology   exam_type\t\tradiology  
0                         NaN                      NaN  
1                         NaN                      NaN  
2                         NaN                      NaN  
3                         NaN                      NaN  
4     2016-05-11 00:00:00.000  XR Chest PA/Lateral\t\t  

[5 rows x 66 columns]

In [291]:
cohort1dta.to_csv(join(mypath,"merged_cohort1.csv"),sep="|",index=False)

# Quick note

Differences in dx columns. Need to account for that in the union of the two.

# Below we merge the moved from cohort 2 grouping

In [249]:
# ORDER NOW MATTERS
onlyfiles3editshort = list( onlyfiles2edit[i] for i in [0,1,4,5] )
onlyfiles3editshort

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

In [250]:
list_ = []

get_list(onlyfiles3editshort,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv
iteration number 0
suffix :  _patient
dimensions :  (2243, 6)
iteration number 1
suffix :  _encounter
dimensions :  (396632, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (396632, 260)
iteration number 3
suffix :  radiology
dimensions :  (1275948, 5)


In [252]:
holdingdf = pd.DataFrame()
cohort1_dta_part2 = merge_lists(list_)

0
1
merging first list slice:  1
   patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
0          49       4860489                 O                Completed   

  visit_status_2_encounter visit_status_3_encounter visit_status_4_encounter  \
0                      NaN                      NaN                      NaN   

      adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
0  2012-07-17 00:00:00.000  2012-07-17 00:00:00.000  2012-07-17 00:00:00.000   

      end_date_d_encounter     admit_source_1_encounter  \
0  2012-07-17 00:00:00.000  Clinic or Physicians Office   

  admit_source_2_encounter admit_source_3_encounter disch_disp_1_encounter  \
0                      NaN                      NaN                    NaN   

  disch_disp_2_encounter disch_disp_3_encounter  zip_d_encounter  \
0                    NaN                    NaN           113043   

  enc_fin_class_encounter  
0                Medicare  
2
merging next list slice: 

In [253]:
cohort1_dta_part2[cohort1_dta_part2['patient_id']==114]

patient_id  age_at_last_encounter_patient gender_patient race_patient  \
0          114                           66.0              F        White   
1          114                           66.0              F        White   
2          114                           66.0              F        White   
3          114                           66.0              F        White   
4          114                           66.0              F        White   
5          114                           66.0              F        White   
6          114                           66.0              F        White   
7          114                           66.0              F        White   
8          114                           66.0              F        White   
9          114                           66.0              F        White   
10         114                           66.0              F        White   
11         114                           66.0              F        White   
12         114                           66.0              F        White   
13         114                           66.0              F        White   
14         114                           66.0              F        White   
15         114                           66.0              F        White   
16         114                           66.0              F        White   
17         114                           66.0              F        White   
18         114                           66.0              F        White   
19         114                           66.0              F        White   
20         114                           66.0              F        White   
21         114                           66.0              F        White   
22         114                           66.0              F        White   
23         114                           66.0              F        White   
24         114                           66.0              F        White   
25         114                           66.0              F        White   
26         114                           66.0              F        White   
27         114                           66.0              F        White   
28         114                           66.0              F        White   
29         114                           66.0              F        White   
30         114                           66.0              F        White   
31         114                           66.0              F        White   
32         114                           66.0              F        White   
33         114                           66.0              F        White   
34         114                           66.0              F        White   
35         114                           66.0              F        White   
36         114                           66.0              F        White   
37         114                           66.0              F        White   
38         114                           66.0              F        White   
39         114                           66.0              F        White   
40         114                           66.0              F        White   
41         114                           66.0              F        White   
42         114                           66.0              F        White   
43         114                           66.0              F        White   
44         114                           66.0              F        White   

         ethnicity_patient date_of_death_d_patient  encounter_id  \
0   Not Hispanic or Latino                     NaN       4645531   
1   Not Hispanic or Latino                     NaN       5066736   
2   Not Hispanic or Latino                     NaN       5066736   
3   Not Hispanic or Latino                     NaN       5097355   
4   Not Hispanic or Latino                     NaN       6214618   
5   Not Hispanic or Latino                     NaN   

In [254]:
cohort1_dta_part2.head()

patient_id  age_at_last_encounter_patient gender_patient race_patient  \
0         114                           66.0              F        White   
1         114                           66.0              F        White   
2         114                           66.0              F        White   
3         114                           66.0              F        White   
4         114                           66.0              F        White   

        ethnicity_patient date_of_death_d_patient  encounter_id  \
0  Not Hispanic or Latino                     NaN       4645531   
1  Not Hispanic or Latino                     NaN       5066736   
2  Not Hispanic or Latino                     NaN       5066736   
3  Not Hispanic or Latino                     NaN       5097355   
4  Not Hispanic or Latino                     NaN       6214618   

  enc_eio_encounter visit_status_1_encounter visit_status_2_encounter  \
0                 O                Completed                 Canceled   
1                 O                Completed                      NaN   
2                 O                Completed                      NaN   
3                 O                Completed                      NaN   
4                 O                Completed                 Canceled   

          ...          dx84_rank_dx_info dx_85_dx_info dx_desc_85_dx_info  \
0         ...                        NaN           NaN                NaN   
1         ...                        NaN           NaN                NaN   
2         ...                        NaN           NaN                NaN   
3         ...                        NaN           NaN                NaN   
4         ...                        NaN           NaN                NaN   

  dx85_rank_dx_info dx_86_dx_info dx_desc_86_dx_info dx86_rank_dx_info  \
0               NaN           NaN                NaN               NaN   
1               NaN           NaN                NaN               NaN   
2               NaN           NaN                NaN               NaN   
3               NaN           NaN                NaN               NaN   
4               NaN           NaN                NaN               NaN   

  order_proc_id_deidradiology ordering_date_offradiology   exam_typeradiology  
0                    856883.0    2013-08-22 00:00:00.000  XR Chest PA/Lateral  
1                    685083.0    2013-08-22 00:00:00.000  XR Chest PA/Lateral  
2                    719735.0    2013-08-22 00:00:00.000  XR Chest PA/Lateral  
3                    973439.0    2014-01-11 00:00:00.000  XR Chest PA/Lateral  
4                    852596.0    2014-06-22 00:00:00.000          CT Chest wo  

[5 rows x 285 columns]

We finally have a merged dataset that should be clean of dupes.

In [255]:
cohort1_dta_part2.to_csv(join(mypath,"merged_movedfromcohort1.csv"),sep="|")

# Merge of the first and second cohort1 groupings.

Note that there's two different column sets


In [30]:
pd.set_option('display.max_columns', None)

In [20]:
# if from before. read in csvs

cohort1dta = pd.read_csv(join(mypath,"merged_cohort1.csv"),sep="|")
cohort1_dta_part2=pd.read_csv(join(mypath,"merged_movedfromcohort1.csv"),sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,22,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
cohort1dta[cohort1dta.patient_id==360469].columns

Index(['patient_id', 'age_at_last_encounter_patient', 'gender_patient',
       'race_patient', 'ethnicity_patient', 'date_of_death_d_patient',
       'encounter_id', 'enc_eio_encounter', 'visit_status_1_encounter',
       'visit_status_2_encounter', 'visit_status_3_encounter',
       'visit_status_4_encounter', 'adm_date_d_encounter',
       'disc_date_d_encounter', 'start_date_d_encounter',
       'end_date_d_encounter', 'admit_source_1_encounter',
       'admit_source_2_encounter', 'admit_source_3_encounter',
       'disch_disp_1_encounter', 'disch_disp_2_encounter',
       'disch_disp_3_encounter', 'zip_d_encounter', 'enc_fin_class_encounter',
       'dx_1_dx_info', 'dx_desc_1_dx_info', 'dx1_rank_dx_info', 'dx_2_dx_info',
       'dx_desc_2_dx_info', 'dx2_rank_dx_info', 'dx_3_dx_info',
       'dx_desc_3_dx_info', 'dx3_rank_dx_info', 'dx_4_dx_info',
       'dx_desc_4_dx_info', 'dx4_rank_dx_info', 'dx_5_dx_info',
       'dx_desc_5_dx_info', 'dx5_rank_dx_info', 'dx_6_dx_info',
       'd

In [25]:
cohort1dta.rename(columns={'exam_type\t\tradiology':'exam_typeradiology'}, inplace=True)

In [22]:
cohort1_dta_part2[cohort1_dta_part2.patient_id==2959602].columns

Index(['Unnamed: 0', 'patient_id', 'age_at_last_encounter_patient',
       'gender_patient', 'race_patient', 'ethnicity_patient',
       'date_of_death_d_patient', 'encounter_id', 'enc_eio_encounter',
       'visit_status_1_encounter',
       ...
       'dx84_rank_dx_info', 'dx_85_dx_info', 'dx_desc_85_dx_info',
       'dx85_rank_dx_info', 'dx_86_dx_info', 'dx_desc_86_dx_info',
       'dx86_rank_dx_info', 'order_proc_id_deidradiology',
       'ordering_date_offradiology', 'exam_typeradiology'],
      dtype='object', length=286)

In [27]:
cohort1_dta_part2[cohort1_dta_part2.patient_id==2959602][['patient_id','exam_typeradiology']]

patient_id     exam_typeradiology
59651     2959602            CT Chest wo
59652     2959602            CT Chest wo
59653     2959602       IOE COM CT CHEST
59654     2959602       IOE COM XR CHEST
59655     2959602   IOE COM CT ANG CHEST
59656     2959602       IOE COM CT CHEST
59657     2959602   IOE COM CT ANG CHEST
59658     2959602  XR Port Chest, 1 View
59659     2959602  XR Port Chest, 1 View
59660     2959602       IOE COM XR CHEST
59661     2959602    XR Chest PA/Lateral
59662     2959602            CT Chest wo
59663     2959602    XR Chest PA/Lateral
59664     2959602            CT Chest wo

In [36]:
cohort1total = pd.merge(cohort1dta,cohort1_dta_part2,how="outer")

In [37]:
cohort1total.to_csv(join(mypath,"merged_cohort1_withcohort2.csv"),sep="|")

# Now merge for the cohort2 sample group

In [262]:
# ORDER NOW MATTERS
onlyfilessampled = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfilessampled = list( onlyfilessampled[i] for i in [10,6,14,15] )
onlyfilessampled

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

In [272]:
patientlist = np.unique(cohort2sampdf.PATIENT_ID)
patientlist[0:5]

array([ 99, 195, 219, 256, 560], dtype=int64)

In [273]:
# memory errors still. merge the first part, return as object, try merge that way?

frame = pd.DataFrame()
list_ = []

def get_list(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)]
            list_.append(df)
    suffix = ('_patient', '_encounter','_dx_info','radiology')
    for i in range(0,len(list_)):
        print("iteration number",i)
        print("suffix : ",suffix[i])
        if i == 0:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0])
            list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)
        else:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0:2])
            list_[i] = list_[i].iloc[:,2:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)

In [274]:
get_list(onlyfilessampled,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv
iteration number 0
suffix :  _patient
dimensions :  (30000, 6)
iteration number 1
suffix :  _encounter
dimensions :  (95743, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (95743, 260)
iteration number 3
suffix :  radiology
dimensions :  (307208, 5)


In [276]:
holdingdf = pd.DataFrame()
cohort2dta = merge_lists(list_)

0
1
merging first list slice:  1
    patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
21          99       5169683                 O                Completed   

   visit_status_2_encounter visit_status_3_encounter visit_status_4_encounter  \
21                      NaN                      NaN                      NaN   

       adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
21  2013-04-06 00:00:00.000  2013-04-06 00:00:00.000  2013-04-06 00:00:00.000   

       end_date_d_encounter     admit_source_1_encounter  \
21  2013-04-06 00:00:00.000  Clinic or Physicians Office   

   admit_source_2_encounter admit_source_3_encounter disch_disp_1_encounter  \
21                      NaN                      NaN                    NaN   

   disch_disp_2_encounter disch_disp_3_encounter  zip_d_encounter  \
21                    NaN                    NaN           149109   

     enc_fin_class_encounter  
21  Blue Cross / Blue Shield  
2
merging

In [277]:
cohort2dta.head()

patient_id  age_at_last_encounter_patient gender_patient  \
0     1843306                           76.0              F   
1     1843306                           76.0              F   
2     1843306                           76.0              F   
3     1843306                           76.0              F   
4     1843306                           76.0              F   

             race_patient       ethnicity_patient  date_of_death_d_patient  \
0  Black/African-American  Not Hispanic or Latino  2015-02-22 00:00:00.000   
1  Black/African-American  Not Hispanic or Latino  2015-02-22 00:00:00.000   
2  Black/African-American  Not Hispanic or Latino  2015-02-22 00:00:00.000   
3  Black/African-American  Not Hispanic or Latino  2015-02-22 00:00:00.000   
4  Black/African-American  Not Hispanic or Latino  2015-02-22 00:00:00.000   

   encounter_id enc_eio_encounter visit_status_1_encounter  \
0       7872100                 I                Completed   
1       7872100                 I                Completed   
2       7872100                 I                Completed   
3       7872100                 I                Completed   
4       7872100                 I                Completed   

  visit_status_2_encounter              ...              dx84_rank_dx_info  \
0                 Canceled              ...                            NaN   
1                 Canceled              ...                            NaN   
2                 Canceled              ...                            NaN   
3                 Canceled              ...                            NaN   
4                 Canceled              ...                            NaN   

  dx_85_dx_info dx_desc_85_dx_info dx85_rank_dx_info dx_86_dx_info  \
0           NaN                NaN               NaN           NaN   
1           NaN                NaN               NaN           NaN   
2           NaN                NaN               NaN           NaN   
3           NaN                NaN               NaN           NaN   
4           NaN                NaN               NaN           NaN   

  dx_desc_86_dx_info dx86_rank_dx_info order_proc_id_deidradiology  \
0                NaN               NaN                    296814.0   
1                NaN               NaN                    315456.0   
2                NaN               NaN                    362063.0   
3                NaN               NaN                    395686.0   
4                NaN               NaN                    493692.0   

  ordering_date_offradiology            exam_typeradiology  
0    2013-10-11 00:00:00.000         XR Port Chest, 1 View  
1    2013-10-11 00:00:00.000                    CT Head wo  
2    2013-10-11 00:00:00.000        CT Neck Soft Tissue wo  
3    2013-10-11 00:00:00.000                    CT Head wo  
4    2013-10-11 00:00:00.000  CT Chest, Abdomen, Pelvis wo  

[5 rows x 285 columns]

In [278]:
cohort2dta.to_csv(join(mypath,"merged_cohort2_sampled.csv"),sep="|")

# Joining to rad notes  

string dict is "encounter_id" and "order_proc_id_deid"

In [3]:
# read csvs if skipping above
cohort1total=pd.read_csv("Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv",sep="|")
cohort2dta=pd.read_csv("Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv",sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,21,22,116,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,11,12,18,19,21,22,37,40,43,46,49,52,55,58,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185

In [4]:
# extract encounter_id from notes
mypathnotes = "Z:/notes v2/"
onlyfilesnotes = [join(mypathnotes,f) for f in listdir(mypathnotes) if isfile(join(mypathnotes, f))]
onlyfilesnotes_sorted = sorted(onlyfilesnotes, key=lambda x:(x!="cohort", x))
print(onlyfilesnotes_sorted)

['Z:/notes v2/cohort1_deid_df.csv', 'Z:/notes v2/cohort1_deid_df_editedforchecks.xlsx', 'Z:/notes v2/cohort2_deid_df.csv', 'Z:/notes v2/cohort_1_v2.zip', 'Z:/notes v2/cohort_2_v2.zip', 'Z:/notes v2/parsing_draft_v2.py']


In [5]:
# cohort 1
notes1 = pd.read_csv(onlyfilesnotes_sorted[0],index_col=None, header=0,sep=",")
notes1.columns = [x.lower() for x in notes1.columns]

In [6]:
# cohort 2
notes2 = pd.read_csv(onlyfilesnotes_sorted[2],index_col=None, header=0,sep=",",error_bad_lines=False)
notes2.columns = [x.lower() for x in notes2.columns]

In [8]:
# split 1
notes1[['encounter_id','order_proc_id_deid']] = notes1['patient'].str.split('_',expand=True)

# split2
notes2[['encounter_id','order_proc_id_deid']] = notes2['patient'].str.split('_',expand=True)

In [21]:
# ensure int type
results = [int(i) for i in notes1.encounter_id]
notes1.encounter_id = results

resultsorder = [int(i) for i in notes1.order_proc_id_deid]
notes1.order_proc_id_deid = resultsorder

In [ ]:
# ensure int type
results = [int(i) for i in notes2.encounter_id]
notes2.encounter_id = results

resultsorder = [int(i) for i in notes2.order_proc_id_deid]
notes2.order_proc_id_deid = resultsorader

In [22]:
notes1[notes1.encounter_id==5035743]

unnamed: 0              scantype                scandate  \
0           0  XR CHEST PA/LATERAL,  2014-10-18**]> 9:58 AM   

                                            clininfo            technique  \
0  **NAME<BBB> **AGE<in 60s> years old. Status po...  XR CHEST PA/LATERAL   

                  comparison  \
0  **DATE<[**2014-09-20**]>.   

                                            findings  \
0  No pneumothorax.  Irregular opacity along the ...   

                                         impressions  \
0  No pneumothorax.  Nonspecific air space opacit...   

                                             elecsig           patient  \
0  **DATE<[**2014-10-18**]> 12:13 PM [**Doctor Fi...   5035743_1018855   

                                             rawtext  encounter_id  \
0  XR CHEST PA/LATERAL, **DATE<[**2014-10-18**]> ...       5035743   

   order_proc_id_deid  
0             1018855

In [26]:
print(cohort1total[cohort1total.encounter_id==5035743].columns)

Index(['Unnamed: 0', 'patient_id', 'age_at_last_encounter_patient',
       'gender_patient', 'race_patient', 'ethnicity_patient',
       'date_of_death_d_patient', 'encounter_id', 'enc_eio_encounter',
       'visit_status_1_encounter',
       ...
       'dx83_rank_dx_info', 'dx_84_dx_info', 'dx_desc_84_dx_info',
       'dx84_rank_dx_info', 'dx_85_dx_info', 'dx_desc_85_dx_info',
       'dx85_rank_dx_info', 'dx_86_dx_info', 'dx_desc_86_dx_info',
       'dx86_rank_dx_info'],
      dtype='object', length=287)


# KC NOTE

Still not matching on the order id. Not sure if missign something there.

### Now join notes for cohort1

In [ ]:
cohort1_dta_notes = pd.merge(cohort1total, notes1, on = 'encounter_id',how="left")
# cohort1_dta_notes = pd.merge(cohort1_dta_notes, notes2, on = 'encounter_id',how="left")
# pd.merge(cohort1total[cohort1total.encounter_id==5035743], notes1, on = 'encounter_id',how="left")[['patient_id','encounter_id','impressions']]

In [28]:
cohort1_dta_notes

Unnamed: 0  patient_id  age_at_last_encounter_patient gender_patient  \
0               0       32124                           90.0              F   
1               1       74912                           63.0              F   
2               2       81545                           65.0              M   
3               3       93110                           47.0              F   
4               4      118647                           52.0              F   
5               5      122487                           43.0              M   
6               6      122522                           74.0              M   
7               7      136921                           57.0              F   
8               8      137154                           54.0              F   
9               9      140791                           50.0              M   
10             10      140791                           50.0              M   
11             11      143528                           72.0              F   
12             12      143528                           72.0              F   
13             13      148260                           71.0              M   
14             14      158414                           66.0              F   
15             15      174454                           83.0              M   
16             16      174454                           83.0              M   
17             17      174454                           83.0              M   
18             18      174454                           83.0              M   
19             19      239207                           32.0              F   
20             20      239207                           32.0              F   
21             21      335825                           42.0              F   
22             22      360469                           65.0              M   
23             23      360469                           65.0              M   
24             24      360469                           65.0              M   
25             25      360469                           65.0              M   
26             26      360469                           65.0              M   
27             27      360469                           65.0              M   
28             28      498795                           59.0              M   
29             29      498795                           59.0              M   
...           ...         ...                            ...            ...   
59988       59948     2740460                           40.0              M   
59989       59949     2740460                           40.0              M   
59990       59950     2740460                           40.0              M   
59991       59951     2740460                           40.0              M   
59992       59952     2740460                           40.0              M   
59993       59953     2740460                           40.0              M   
59994       59954     2740460                           40.0              M   
59995       59955     2740460                           40.0              M   
59996       59956     2740460                           40.0              M   
59997       59957     2740460                           40.0              M   
59998       59958     2779581                           74.0              M   
59999       59959     2779581                           74.0              M   
60000       59960     2779581                           74.0              M   
60001       59961     2849686                           56.0              M   
60002       59962     2849686                           56.0              M   
60003       59963     2849686                           56.0              M   
60004       59964     2959602                           64.0              M   
60005       59965     2959602                           64.0              M   
60006       59966     2959602                           6

In [ ]:
cohort1_dta_notes[cohort1_dta_notes.impressions.notnull()]

### Now join notes for cohort2

In [29]:
cohort2_dta_notes = pd.merge(cohort2dta, notes2, on = 'encounter_id',how="left")

KeyboardInterrupt: 

In [376]:
cohort2_dta_notes[cohort2_dta_notes.impressions.notnull()]

Empty DataFrame
Columns: [patient_id, age_at_last_encounter_patient, gender_patient, race_patient, ethnicity_patient, date_of_death_d_patient, encounter_id_encounter, enc_eio_encounter, visit_status_1_encounter, visit_status_2_encounter, visit_status_3_encounter, visit_status_4_encounter, adm_date_d_encounter, disc_date_d_encounter, start_date_d_encounter, end_date_d_encounter, admit_source_1_encounter, admit_source_2_encounter, admit_source_3_encounter, disch_disp_1_encounter, disch_disp_2_encounter, disch_disp_3_encounter, zip_d_encounter, enc_fin_class_encounter, encounter_id_encounter_provider, admitting_prov_id_encounter_provider, admitting_provider_department_encounter_provider, attnd_prov_id_encounter_provider, attending_department_encounter_provider, referring_prov_id_encounter_provider, referring_department_encounter_provider, contact_date_off_encounter_social, is_tobacco_user_encounter_social, tobacco_pak_per_dy_encounter_social, tobacco_used_years_encounter_social, tobacco_comment_encounter_social, smoking_quit_date_off_encounter_social, cigarettes_yn_encounter_social, pipes_yn_encounter_social, cigars_yn_encounter_social, snuff_yn_encounter_social, chew_yn_encounter_social, is_alcohol_user_encounter_social, alcohol_oz_per_wk_encounter_social, is_ill_drug_user_encounter_social, iv_drug_user_yn_encounter_social, illicit_drug_freq_encounter_social, is_sexually_actv_encounter_social, female_partner_yn_encounter_social, male_partner_yn_encounter_social, condom_yn_encounter_social, pill_yn_encounter_social, diaphragm_yn_encounter_social, iud_yn_encounter_social, surgical_yn_encounter_social, spermicide_yn_encounter_social, implant_yn_encounter_social, rhythm_yn_encounter_social, injection_yn_encounter_social, sponge_yn_encounter_social, inserts_yn_encounter_social, abstinence_yn_encounter_social, years_education_encounter_social, tob_src_c_encounter_social, alcohol_src_c_encounter_social, sex_src_c_encounter_social, alcohol_use_c_encounter_social, ill_drug_user_c_encounter_social, sexually_active_c_encounter_social, tobacco_user_encounter_social, smokeless_tob_use_c_encounter_social, smokeless_quit_date_off_encounter_social, smoking_tob_use_encounter_social, unknown_fam_hx_yn_encounter_social, smoking_start_date_off_encounter_social, encounter_id_dx_info, dx_1_dx_info, dx_desc_1_dx_info, dx1_rank_dx_info, dx_2_dx_info, dx_desc_2_dx_info, dx2_rank_dx_info, dx_3_dx_info, dx_desc_3_dx_info, dx3_rank_dx_info, dx_4_dx_info, dx_desc_4_dx_info, dx4_rank_dx_info, dx_5_dx_info, dx_desc_5_dx_info, dx5_rank_dx_info, dx_6_dx_info, dx_desc_6_dx_info, dx6_rank_dx_info, dx_7_dx_info, dx_desc_7_dx_info, dx7_rank_dx_info, dx_8_dx_info, dx_desc_8_dx_info, dx8_rank_dx_info, ...]
Index: []

[0 rows x 119 columns]